# Week 1 — LangChain Fundamentals & Basic Chains

**Course:** LangChain for AI Applications  
**Week Focus:** Understand LLMs, prompts, chains, and how to build your first AI applications.

---

## 🎯 Learning Objectives

By the end of this week, you will:
- Understand LLM fundamentals and API interactions
- Build prompt templates for consistent outputs
- Create simple and sequential chains
- Handle LLM responses and parse outputs
- Build your first customer support chatbot
- Deploy a basic LangChain application

## 📊 Real-World Context

LangChain lets you build AI applications that:
- **Automate tasks**: Customer support, data processing, content generation
- **Augment workflows**: Combine LLMs with your tools and data
- **Scale reasoning**: Chain multiple LLM calls for complex problems
- **Maintain context**: Track conversation history and state

Companies use LangChain to:
- Build AI customer service agents (Intercom, Zendesk integration)
- Automate document processing (PDFs → insights)
- Generate personalized content (emails, recommendations)
- Create code assistants and debugging tools

## 🏢 Scenario — Build an AI Customer Support Assistant

Your SaaS company receives 500+ support tickets daily. Your goal:
1. Analyze customer issues automatically
2. Draft responses using LLMs
3. Route complex issues to humans
4. Track conversation context

We'll build this step-by-step using LangChain.

## 📚 Key Concepts — Why LangChain?

### The Problem Without LangChain
```python
# Raw OpenAI API (verbose, error-prone)
import openai

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a support agent"},
        {"role": "user", "content": "My app is crashing"}
    ],
    temperature=0.7,
)
answer = response['choices'][0]['message']['content']
```

### The Solution With LangChain
```python
# LangChain (clean, composable, extensible)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4")
prompt = ChatPromptTemplate.from_template(
    "You are a support agent. Help: {user_issue}"
)
chain = prompt | llm
answer = chain.invoke({"user_issue": "My app is crashing"})
```

### LangChain's Core Concepts
1. **LLMs**: Language models (OpenAI, Anthropic, Llama, etc.)
2. **Prompts**: Templates for consistent instructions
3. **Chains**: Sequences of operations (prompt → LLM → parser)
4. **Memory**: Tracking conversation history
5. **Tools**: Integrations (databases, APIs, files)
6. **Agents**: AI making decisions about which tools to use

## ✍️ Hands-on Exercises

### Exercise 1: Prompt Templates
Create a prompt template for support tickets:
- Input: customer_name, issue_type, issue_description
- Output: helpful support response
- Test with multiple customer scenarios

### Exercise 2: Simple Chain
Build a chain: Prompt → LLM → Output Parser
- Use ChatOpenAI model
- Parse structured output (JSON)
- Handle multiple issue types

### Exercise 3: Chain Pipeline
Chain multiple operations:
1. Analyze issue (classify as bug/feature/question)
2. Generate response based on type
3. Rate urgency (1-5)
4. Return structured result

<details>
<summary>💡 Hint — Building Your First Chain</summary>

**Step 1: Install LangChain**
```bash
pip install langchain langchain-openai python-dotenv
```

**Step 2: Set up environment**
```python
import os
from dotenv import load_dotenv

load_dotenv()  # Load OPENAI_API_KEY from .env
```

**Step 3: Create LLM**
```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0.7)
```

**Step 4: Build prompt + chain**
```python
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "You are a support agent. {input}"
)
chain = prompt | llm
result = chain.invoke({"input": "...user question..."})
```

</details>

<details>
<summary>✅ Solution — Customer Support Chain</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define expected output structure
class SupportResponse(BaseModel):
    issue_type: str = Field(description="bug, feature, or question")
    response: str = Field(description="helpful support message")
    urgency: int = Field(description="1-5 scale")
    escalate: bool = Field(description="needs human review?")

# Create LLM and parser
llm = ChatOpenAI(model="gpt-4", temperature=0.7)
parser = JsonOutputParser(pydantic_object=SupportResponse)

# Build prompt with format instructions
prompt = ChatPromptTemplate.from_template(
    """You are a SaaS support agent. Analyze this customer issue:
    
    Customer: {customer_name}
    Issue: {issue_description}
    
    Provide a response with:
    - Issue classification (bug/feature/question)
    - Helpful response text
    - Urgency level (1-5)
    - Whether to escalate to human
    
    {format_instructions}"""
)

# Add format instructions
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Build chain
chain = prompt | llm | parser

# Test
result = chain.invoke({
    "customer_name": "Alice",
    "issue_description": "The export button doesn't work"
})

print(f"Type: {result['issue_type']}")
print(f"Response: {result['response']}")
print(f"Urgency: {result['urgency']}")
```

**Why this works:**
- `ChatPromptTemplate`: Structured prompts with variable injection
- `JsonOutputParser`: Automatic JSON parsing with validation
- Pipe operator `|`: Clean chain composition (prompt → LLM → parser)
- `pydantic`: Type-safe output validation

</details>

## 🤔 Reflection & Application

**Question 1:** When should you use temperature=0.7 vs 0 vs 1?
- temperature=0: Deterministic, best for classification and extraction
- temperature=0.7: Balanced creativity and consistency (default good choice)
- temperature=1.0: Creative, good for brainstorming and content generation

**Question 2:** How do you prevent hallucinations?
- Use structured outputs (JSON parsing)
- Add "Answer only based on provided information"
- Use few-shot examples in prompts
- Verify against ground truth

**Question 3:** When is a chain not enough?
- When you need to make decisions (use agents)
- When you need to retry/recover (add error handling)
- When you need tool access (use agents with tools)

## 📝 Practice Assignment

**Build:** An automated email classification system

Requirements:
1. Take email subject + body as input
2. Classify: spam, urgent, normal, followup
3. Extract action items (if any)
4. Generate brief summary
5. Test with 5 different emails

## 🔗 Next Steps

In Week 2, we'll add **memory** to track conversations and build **agents** that can use tools autonomously.